# Python Study Group : Determining Codon Degeneracy
## March 28th, 2019

We will be measuring codon degeneracy using a protein between 3 related organisms.

Once we have measured the codon usage for each of these organisms, we may begin to investigate any biases one organism may have for any particular codon. This ultimately allows scientists to detect purifying selection and a number of metrics have been implemented to identify these selection events.

Here is a nice review with more detailed explanations: [Nat. Rev. Gen. 2011](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074964/ "Synonymous but not the same: the causes and consequences of codon bias")

### Some common terms:

- non-synonymous to synonymous codon subsitutions. (dN/dS)
- Relative synonymous codon usage (RSCU)

### Setup

1. Clone the GitHub repo for [PythonStudyGroup](https://github.com/ComBEE-UW-Madison/PythonStudyGroup "PSG Repo")
2. Download & Install [Docker](docker.com "docker website")
3. pull the docker images

```bash
# 1. Clone the GitHub Repo
git clone https://github.com/ComBEE-UW-Madison/PythonStudyGroup
# 3. Pull the docker images to run bioinformatic tools
docker pull evanrees/kwanlab:pal2nal
docker pull biocontainers/clustal-omega:v1.2.1_cv5
```

### Begin

We will start out by first ensuring we have all of the necessary data. I've placed the data in the [PythonStudyGroup](https://github.com/ComBEE-UW-Madison/PythonStudyGroup "PSG Repo") repository and you can locate it in the directory titled `/path/to/PSG/repo/2019Spring/mar28`.



## Calculating Non-synonymous to Synonymous codon usage (dN/dS)

### General workflow

1. Concatenate **protein** sequences
2. Concatenate nucleotide sequences in the _same order_ of the concatenated protein sequences
3. Align protein sequences using `clustalo`
4. Use `pal2nal.pl` to perform protein to nucleotide alignment with aligned
proteins and concatenated nucleotide files
5. Use `codeml` with a respective `*.cnt` file and a corresponding `*.tree` file
with simple newick format *(<gene_name>,arbitrary_name);*

In [1]:
!ls example/data/*

example/data/processed:

example/data/raw:
org1_yenB.faa org2_yenB.faa org3_yenB.faa
org1_yenB.fna org2_yenB.fna org3_yenB.fna


In [2]:
# 1. Concatenate **amino acid** sequences
# !rm example/data/processed/orgs_yenB.faa
!echo "previous to concatenation"
!ls example/data/raw/*.faa
!cat example/data/raw/*.faa >> example/data/processed/orgs_yenB.faa
!echo "Post concatenation"
!ls example/data/processed/*.faa

previous to concatenation
example/data/raw/org1_yenB.faa example/data/raw/org3_yenB.faa
example/data/raw/org2_yenB.faa
Post concatenation
example/data/processed/orgs_yenB.faa


In [3]:
# Check sequence order
!grep ">" example/data/processed/orgs_yenB.faa

>example1_hypothetical_protein
>example2_hypothetical_protein
>example3_hypothetical_protein


In [4]:
# 2. concatenate **nucleotide** sequences in corresponding order of orgs_yenB.faa
# !rm example/data/processed/orgs_yenB.fna
!echo "previous to concatenation"
!ls example/data/raw/*.fna
!cat example/data/raw/*.fna >> example/data/processed/orgs_yenB.fna
!echo "Post concatenation"
!ls example/data/*

previous to concatenation
example/data/raw/org1_yenB.fna example/data/raw/org3_yenB.fna
example/data/raw/org2_yenB.fna
Post concatenation
example/data/processed:
orgs_yenB.faa orgs_yenB.fna

example/data/raw:
org1_yenB.faa org2_yenB.faa org3_yenB.faa
org1_yenB.fna org2_yenB.fna org3_yenB.fna


In [5]:
# Check the sequence order
!grep ">" example/data/processed/orgs_yenB.fna

>example1_hypothetical_protein
>example2_hypothetical_protein
>example3_hypothetical_protein


In [6]:
!echo "nucleotides file"
!grep ">" example/data/processed/orgs_yenB.fna
!echo "amino acids file"
!grep ">" example/data/processed/orgs_yenB.faa

nucleotides file
>example1_hypothetical_protein
>example2_hypothetical_protein
>example3_hypothetical_protein
amino acids file
>example1_hypothetical_protein
>example2_hypothetical_protein
>example3_hypothetical_protein


In [7]:
!docker run --rm biocontainers/clustal-omega:v1.2.1_cv5 clustalo -h

Clustal Omega - 1.2.1 (AndreaGiacomo)

If you like Clustal-Omega please cite:
 Sievers F, Wilm A, Dineen D, Gibson TJ, Karplus K, Li W, Lopez R, McWilliam H, Remmert M, Söding J, Thompson JD, Higgins DG.
 Fast, scalable generation of high-quality protein multiple sequence alignments using Clustal Omega.
 Mol Syst Biol. 2011 Oct 11;7:539. doi: 10.1038/msb.2011.75. PMID: 21988835.
If you don't like Clustal-Omega, please let us know why (and cite us anyway).

Check http://www.clustal.org for more information and updates.

Usage: clustalo [-hv] [-i {<file>,-}] [--hmm-in=<file>]... [--dealign] [--profile1=<file>] [--profile2=<file>] [--is-profile] [-t {Protein, RNA, DNA}] [--infmt={a2m=fa[sta],clu[stal],msf,phy[lip],selex,st[ockholm],vie[nna]}] [--distmat-in=<file>] [--distmat-out=<file>] [--guidetree-in=<file>] [--guidetree-out=<file>] [--pileup] [--full] [--full-iter] [--cluster-size=<n>] [--clustering-out=<file>] [--trans=<n>] [--posterior-out=<file>] [--use-kimura] [--percent-id] [-o {f

In [9]:
#!docker run --rm -it biocontainers/clustal-omega:v1.2.1_cv5
!echo "Pre-alignment"
!ls example/data/processed
!docker run \
    --rm \
    -v $(pwd)/example/data:/output \
    biocontainers/clustal-omega:v1.2.1_cv5 \
        clustalo -i /output/processed/orgs_yenB.faa \
        --force \
        --auto -o /output/processed/orgs_yenB.aln
!echo "Post-alignment"        
!ls example/data/processed

Pre-alignment
orgs_yenB.aln orgs_yenB.faa orgs_yenB.fna
Post-alignment
orgs_yenB.aln orgs_yenB.faa orgs_yenB.fna


### Running pal2nal (protein alignment to nucleotide alignment)

Now that we have aligned our protein sequences we need to perform a protein alignment to nucleotide alignment calculation to determine the corresponding codons.

Luckily for us, I've constructed a docker image to manage all of the dependencies of running it!

This works similarly to the clustalo docker image...

In [10]:
!docker run --rm evanrees/kwanlab:pal2nal pal2nal.pl -h


pal2nal.pl  (v14)

Usage:  pal2nal.pl  pep.aln  nuc.fasta  [nuc.fasta...]  [options]


    pep.aln:    protein alignment either in CLUSTAL or FASTA format

    nuc.fasta:  DNA sequences (single multi-fasta or separated files)

    Options:  -h            Show help 

              -output (clustal|paml|fasta|codon)
                            Output format; default = clustal

              -blockonly    Show only user specified blocks
                            '#' under CLUSTAL alignment (see example)

              -nogap        remove columns with gaps and inframe stop codons

              -nomismatch   remove mismatched codons (mismatch between
                            pep and cDNA) from the output

              -codontable  N
                    1  Universal code (default)
                    2  Vertebrate mitochondrial code
                    3  Yeast mitochondrial code
                    4  Mold, Protozoan, and Coelenterate Mitochondrial code
                       and M

In [11]:
!docker run \
    --rm \
    -v $(pwd)/example/data:/output \
    evanrees/kwanlab:pal2nal \
        pal2nal.pl /output/processed/orgs_yenB.aln /output/processed/orgs_yenB.fna

CLUSTAL W multiple sequence alignment

example1_hypothetical_protein    ATGAGTGAATTGCCTTCTAATAAAGCAGTTGATAATATCGATGCCTCTGTATTAGCTACT
example2_hypothetical_protein    ATGAGTGAATCGTCTTCTAATAAAGTGGTTGATAATGTCAGTTATTCTGTATTACCCACT
example3_hypothetical_protein    ATGAATGAACCCTCTCCTGATAAAACGGTTGATAATATAAGCTTTTCTGAGTTGCCCGTT

example1_hypothetical_protein    CAATCACCGAATTTATCACACCGTTTACGTATGCTGATATGCACACTGTTATTAGCTGTC
example2_hypothetical_protein    CAGCAGTCAAAGTTATTGCACAGTATGCGTACGCTTTTATCCACACTATTATTAGCTATC
example3_hypothetical_protein    CAACAGCCAAGCCTACTGCACCGTATTCGTACACTTGTATCTGCACTATTGTTAGCTATC

example1_hypothetical_protein    TGGTGTTTGGTGATGCCATTGATTAATCTAGTCTGGCGTTTGCTTCATATCCCATACCTG
example2_hypothetical_protein    TGGTGTTTGGTGATGCCATTAATTAATGTAGTCTGGCGTTTGCTTCATATACCGCATCTT
example3_hypothetical_protein    TGGTGTTTGGTAATGCCTTTGGTTAATTTGGTTTGGCGTCTGCTTTATATCCCACACCTT

example1_hypothetical_protein    GAGAAATGTTATTTTATTTTTCACGGTGGATGTTGTTGGCTATTTAATTTGCGTTGCATT
example2_hypotheti

In [12]:
!docker run \
    --rm \
    -v $(pwd)/example/data:/output \
    evanrees/kwanlab:pal2nal \
        pal2nal.pl /output/processed/orgs_yenB.aln \
        /output/processed/orgs_yenB.fna \
        -output codon

                                 M   S   E   L   P   S   N   K   A   V   D   N   I   D   A   S   V   L   A   T
example1_hypothetical_protein    ATG AGT GAA TTG CCT TCT AAT AAA GCA GTT GAT AAT ATC GAT GCC TCT GTA TTA GCT ACT
                                 M   S   E   S   S   S   N   K   V   V   D   N   V   S   Y   S   V   L   P   T
example2_hypothetical_protein    ATG AGT GAA TCG TCT TCT AAT AAA GTG GTT GAT AAT GTC AGT TAT TCT GTA TTA CCC ACT
                                 M   N   E   P   S   P   D   K   T   V   D   N   I   S   F   S   E   L   P   V
example3_hypothetical_protein    ATG AAT GAA CCC TCT CCT GAT AAA ACG GTT GAT AAT ATA AGC TTT TCT GAG TTG CCC GTT

                                 Q   S   P   N   L   S   H   R   L   R   M   L   I   C   T   L   L   L   A   V
example1_hypothetical_protein    CAA TCA CCG AAT TTA TCA CAC CGT TTA CGT ATG CTG ATA TGC ACA CTG TTA TTA GCT GTC
                                 Q   Q   S   K   L   L   H   S   M   R   T   L   L   S   T   L   L   L 

In [17]:
!docker run \
    --rm \
    -v $(pwd)/example/data:/output \
    evanrees/kwanlab:pal2nal \
        pal2nal.pl /output/processed/orgs_yenB.aln /output/processed/orgs_yenB.fna \
        -output paml \
        -nogap \
        -codontable 11 \
        > example/orgs_yenB.paml
        #We have now exited the docker container directing the stdout to the localhost path 

#------------------------------------------------------------------------#
#  Input files:  /output/processed/orgs_yenB.aln /output/processed/orgs_yenB.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#



In [36]:
# Let's check to make sure we have the file we want
!ls example/

data      test.cnt  test.tree


In [20]:
!cat example/test.cnt

      seqfile = test.codon
     treefile = test.tree
      outfile = test.codeml

        noisy = 0   * 0,1,2,3,9: how much rubbish on the screen
      verbose = 0   * 1: detailed output, 0: concise output
      runmode = -2  * 0: user tree;  1: semi-automatic;  2: automatic
                    * 3: StepwiseAddition; (4,5):PerturbationNNI; -2: pairwise

    cleandata = 1   * "I added on 07/07/2004" Mikita Suyama

      seqtype = 1   * 1:codons; 2:AAs; 3:codons-->AAs
    CodonFreq = 2   * 0:1/61 each, 1:F1X4, 2:F3X4, 3:codon table
        model = 2
                    * models for codons:
                        * 0:one, 1:b, 2:2 or more dN/dS ratios for branches

      NSsites = 0   * dN/dS among sites. 0:no variation, 1:neutral, 2:positive
        icode = 0   * 0:standard genetic code; 1:mammalian mt; 2-10:see below
        Mgene = 0   * 0:rates, 1:separate; 2:pi, 3:kappa, 4:all

    fix_kappa = 0   * 1: kappa fixed, 0: kappa to be estimated

    fix_omega = 0   * 1: omega or omega_1 

### Running `codeml` to determine dN/dS:

`codeml` will use a file in which you may specify the parameters..
We will take the more difficult approach here... Because we want to be able to extend this analysis to an entire genome's protein set.

**Note:** Parameters for `codeml` are prescribed in `example/test.cnt`

#### These are the first few lines in the test.cnt file that need to be altered.

- seqfile = test.codon
- treefile = test.tree
- outfile = test.codeml

In [21]:
import re

outfile = open('example/yenB.cnt', 'w')
with open('example/test.cnt') as fh:
    for line in fh:
        match = re.search(r'seqfile = test.codon|outfile = test.codeml|treefile = test.tree',line)
        if match:
            line = line.replace('test.codon','orgs_yenB.paml')
            line = line.replace('test.tree','orgs_yenB.tree')
            line = line.replace('test.codeml','orgs_yenB.codeml')
        outfile.write(line)
            
outfile.close()            

In [22]:
# Let's check that we have properly configured our file...
!head example/yenB.cnt

      seqfile = orgs_yenB.paml
     treefile = orgs_yenB.tree
      outfile = orgs_yenB.codeml

        noisy = 0   * 0,1,2,3,9: how much rubbish on the screen
      verbose = 0   * 1: detailed output, 0: concise output
      runmode = -2  * 0: user tree;  1: semi-automatic;  2: automatic
                    * 3: StepwiseAddition; (4,5):PerturbationNNI; -2: pairwise

    cleandata = 1   * "I added on 07/07/2004" Mikita Suyama


### Constructing the required tree file:

Now that we have our properly formatted config file: `example/yenB.cnt`
we only need our tree file as input and we can finally run `codeml`.

The tree file can be simple if we are simply doing a comparison between already identified homologous genes.

The tree file will take the format of `(gene_of_interest, arbitrary_gene_name);`.

As an example, let's look at the test tree file `example/test.tree`.

In [23]:
!cat example/test.tree

(queryGene,ArbitraryGeneName);


As you can see the tree file is simple!

So let's simply replace the query gene value with our gene of interest _(yenB)_.

In [24]:
import os

InTreeFilePath = os.path.abspath('example/test.tree')
# providing the abspath method from os.path gives us the absolute path to the tree file...
# This can come in handy when operating on a remote file system or trying to extend your program to other users.
# Name the file you want to write out..
OutTreeFilePath = 'example/yenB.tree'
bname, ext = os.path.splitext(os.path.basename(OutTreeFilePath))
# Open the file for writing...
outfile = open(OutTreeFilePath, 'w')
# Open the test.tree file for string replacement and writing to yenB.tree
with open(InTreeFilePath) as fh:
    for line in fh:
        line = line.replace('queryGene', bname)
        outfile.write(line)

outfile.close()

In [25]:
!cat example/yenB.tree

(yenB,ArbitraryGeneName);


### Perform codeml dN/dS calcuations for our gene of interest

We now have our gene's config file `yenB.cnt` and the corresponding tree file `yenB.tree`.

Once again let's run `codeml` using docker...

In [28]:
!docker run \
    --rm \
    -w /output \
    -v $(pwd)/example:/output:rw \
            evanrees/kwanlab:pal2nal codeml yenB.cnt

CODONML in paml version 4.9i, September 2018



In [29]:
#Let's look through our directory and see what files codeml produced
!ls example/

2ML.dN           2NG.dS           orgs_yenB.paml   test.cnt
2ML.dS           2NG.t            rst              test.tree
2ML.t            data             rst1             yenB.cnt
2NG.dN           orgs_yenB.codeml rub              yenB.tree


As you can see, there are a number of files codeml has produced. notice we now have a file with the _codeml_ suffix and file with our dN/dS suffixes!

In [30]:
!cat example/orgs_yenB.codeml

CODONML (in paml version 4.9i, September 2018)  orgs_yenB.paml
Model: several dN/dS ratios for branches, 
Codon frequency model: F3x4
ns =   3  ls = 281

Codon usage in sequences
--------------------------------------------------------------------------------------
Phe TTT  15  16  16 | Ser TCT   7   5   4 | Tyr TAT   8   9  11 | Cys TGT   5   6   5
    TTC   1   1   2 |     TCC   2   3   2 |     TAC   2   0   0 |     TGC   3   2   3
Leu TTA  16  17  10 |     TCA   5   4   3 | *** TAA   0   0   0 | *** TGA   0   0   0
    TTG   8   7   9 |     TCG   1   2   4 |     TAG   0   0   0 | Trp TGG   5   5   5
--------------------------------------------------------------------------------------
Leu CTT   1   3   5 | Pro CCT   6   5   6 | His CAT   2   8   4 | Arg CGT  10   8   9
    CTC   0   1   0 |     CCC   0   3   5 |     CAC   5   1   3 |     CGC   1   1   1
    CTA   4   5   4 |     CCA   4   3   3 | Gln CAA   6   6  11 |     CGA   1   0   1
    CTG   8   3   6 |     CCG   1   2   1 |  

## Woah! 

check it out, we have our dN/dS values in our `orgs_yenB.codeml` file! 

So... we're finished right? Not quite! We want to make this extensible across a whole genome set!

So we will need to grab these values for each comparison and generate our own table!

Let's make this happen:

### Now let us (hypothetically) collate every `*.codeml` file into a table

If you inspect the `*.codeml` file, you may notice two things that are important for collating our data.

1. There is a line containing the compared genes
2. there is a line containing the associated dNdS values.

Let's parse these values and write to a table!

In [31]:
from glob import glob
#Glob is going to grab all of the *codeml files we need!
import re
import os
import pprint

#Grab our codeml files
codeml_fpaths = [os.path.abspath(fp) for fp in glob('example/*.codeml')]
print('Num *.codeml files: {}'.format(len(codeml_fpaths)))
print(codeml_fpaths)

Num *.codeml files: 1
['/Users/rees/Wisc/COMBEE/PythonStudyGroup/2019Spring/mar28/example/orgs_yenB.codeml']


In [32]:
tab_fpath = 'example/genomes_dnds_analysis.tsv'

for fp in codeml_fpaths:
    with open(fp) as fh:
        for index,line in enumerate(fh):
            print(index, line)
            # index value from genes noted for pairwise comparison is 4 lines following
            # i.e. dN/dS values 4 after this line

0 CODONML (in paml version 4.9i, September 2018)  orgs_yenB.paml

1 Model: several dN/dS ratios for branches, 

2 Codon frequency model: F3x4

3 ns =   3  ls = 281

4 

5 Codon usage in sequences

6 --------------------------------------------------------------------------------------

7 Phe TTT  15  16  16 | Ser TCT   7   5   4 | Tyr TAT   8   9  11 | Cys TGT   5   6   5

8     TTC   1   1   2 |     TCC   2   3   2 |     TAC   2   0   0 |     TGC   3   2   3

9 Leu TTA  16  17  10 |     TCA   5   4   3 | *** TAA   0   0   0 | *** TGA   0   0   0

10     TTG   8   7   9 |     TCG   1   2   4 |     TAG   0   0   0 | Trp TGG   5   5   5

11 --------------------------------------------------------------------------------------

12 Leu CTT   1   3   5 | Pro CCT   6   5   6 | His CAT   2   8   4 | Arg CGT  10   8   9

13     CTC   0   1   0 |     CCC   0   3   5 |     CAC   5   1   3 |     CGC   1   1   1

14     CTA   4   5   4 |     CCA   4   3   3 | Gln CAA   6   6  11 |     CGA   1   0 

In [34]:
from Bio import SeqIO

# tab_fpath = 'example/genomes_dnds_analysis.tsv'
concatenated_fasta_fpath = 'example/data/processed/orgs_yenB.faa'
# outfile = open(tab_fpath,'w')
genes = SeqIO.to_dict(SeqIO.parse(concatenated_fasta_fpath, 'fasta'))
print(genes.keys())
# print(genes)

dict_keys(['example1_hypothetical_protein', 'example2_hypothetical_protein', 'example3_hypothetical_protein'])
{'example1_hypothetical_protein': SeqRecord(seq=Seq('MSELPSNKAVDNIDASVLATQSPNLSHRLRMLICTLLLAVWCLVMPLINLVWRL...GGR', SingleLetterAlphabet()), id='example1_hypothetical_protein', name='example1_hypothetical_protein', description='example1_hypothetical_protein', dbxrefs=[]), 'example2_hypothetical_protein': SeqRecord(seq=Seq('MSESSSNKVVDNVSYSVLPTQQSKLLHSMRTLLSTLLLAIWCLVMPLINVVWRL...DSQ', SingleLetterAlphabet()), id='example2_hypothetical_protein', name='example2_hypothetical_protein', description='example2_hypothetical_protein', dbxrefs=[]), 'example3_hypothetical_protein': SeqRecord(seq=Seq('MNEPSPDKTVDNISFSELPVQQPSLLHRIRTLVSALLLAIWCLVMPLVNLVWRL...DDK', SingleLetterAlphabet()), id='example3_hypothetical_protein', name='example3_hypothetical_protein', description='example3_hypothetical_protein', dbxrefs=[])}


In [35]:
ioi = None
comps = dict()
for fp in codeml_fpaths:
    with open(fp) as fh:
        for index,line in enumerate(fh):
            if index == 0:
                codeml_query = line.split()[-1]
                gene = codeml_query.split('.')[0]
                continue
            for gene in genes:
                if gene in line and '...' in line:
#                     print(line.partition('...'))
                    c1, c1_gene, _, c2, c2_gene = line.split()
                    ioi = index + 4
                    break
            if ioi == index:
#                 print(line.split())
                llist = line.split()
                dnds = llist[7]
                dn = llist[10]
                ds = llist[-1]
            try:
                comps.update({'{}:{}'.format(c1,c2):dnds})
            except NameError as err:
                pass
#             print(c1, c2, dnds)

for k,v in comps.items():
    print('pairwise comparison: {} dNdS: {}'.format(k, v))

pairwise comparison: 2:1 dNdS: 0.1676
pairwise comparison: 3:1 dNdS: 0.0946
pairwise comparison: 3:2 dNdS: 0.1566
